In [70]:
import sympy as sp
from sympy import symbols
from sympy.logic.boolalg import And, Implies
from sympy.logic import inference

sp.__version__

'1.13.3'

# Using Sympy

* Major premise: All humans are mortal.
* Minor premise: All Greeks are humans.
* Conclusion/Consequent: All Greeks are mortal.

In [71]:
def propositional_logic_syllogism():
    from sympy import symbols
    from sympy.logic.boolalg import And, Implies
    from sympy.logic import inference

    # Define propositions
    P = symbols("P")  # P: All humans are mortal.
    A = symbols("A")  # Q: All Greeks are humans.
    R = symbols("R")  # R: All Greeks are mortal.

    # Define the premises
    premise1 = P  # All humans are mortal.
    premise2 = A  # All Greeks are humans.

    # Conclusion we wish to verify
    conclusion = R  # All Greeks are mortal.

    # Combine premises using logical conjunction
    premises = And(premise1, premise2)

    # Form the implication from premises to conclusion
    syllogism = Implies(premises, conclusion)

    print("Syllogism in propositional logic:")
    print(inference.satisfiable(syllogism))
propositional_logic_syllogism()

Syllogism in propositional logic:
{R: True, P: False, A: False}


In [82]:
propositions = [
    "A | ~B",
    "C & A",
    "C >> B",
]

global_statement = sp.sympify(
    " & ".join(["(" + str(symp_statement) + ")" for symp_statement in propositions])
)

all_symbols = global_statement.atoms() | global_statement.free_symbols

if inference.valid(global_statement):
    print("Valid in all Worlds")
else:
    models_for_global = inference.satisfiable(global_statement, all_models=True)
    if all(models_for_global):
        print("Valid in some possibilities")
        for model in inference.satisfiable(global_statement, all_models=True):
            print(model if model else "Impossible")
    else:
        print("Invalid in all Worlds")

Valid in some possibilities
{A: True, B: True, C: True}


In [73]:
def predicate_logic_syllogism():
    from sympy import symbols, Function, Implies
    from sympy.logic.boolalg import ForAll

    # Define variables and predicates
    x = symbols("x")
    H = Function("H")  # H(x): x is human
    M = Function("M")  # M(x): x is mortal
    G = Function("G")  # G(x): x is Greek

    # Major premise: For all x, if x is human then x is mortal.
    major_premise = ForAll(x, Implies(H(x), M(x)))

    # Minor premise: For all x, if x is Greek then x is human.
    minor_premise = ForAll(x, Implies(G(x), H(x)))

    # Conclusion: For all x, if x is Greek then x is mortal.
    conclusion = ForAll(x, Implies(G(x), M(x)))

    print("Major premise:")
    print(major_premise)
    print("\nMinor premise:")
    print(minor_premise)
    print("\nConclusion:")
    print(conclusion)

    # Although SymPy does not perform automated theorem proving for predicate logic,
    # we can see that the conclusion logically follows from the premises.
    # By the transitivity of implication:
    # From G(x) ⇒ H(x) and H(x) ⇒ M(x), we deduce G(x) ⇒ M(x).

predicate_logic_syllogism()

ImportError: cannot import name 'ForAll' from 'sympy.logic.boolalg' (c:\Users\rahimi\AppData\Local\miniconda3\envs\logic_env\lib\site-packages\sympy\logic\boolalg.py)

# Using z3 solver

In [36]:
def propositional_logic_syllogism_z3():
    from z3 import Bool, And, Implies, Not, Solver, unsat, prove

    # Define atomic propositions
    P = Bool("P")  # P: All humans are mortal.
    Q = Bool("Q")  # Q: All Greeks are humans.
    R = Bool("R")  # R: All Greeks are mortal.

    # Create a solver
    s = Solver()

    # Add premises (assume P and Q are true)
    s.add(P)
    s.add(Q)

    # Attempt to prove that P and Q imply R
    # In propositional logic, without additional implications, we cannot derive R from P and Q
    # Let's check if (P ∧ Q ∧ ¬R) is satisfiable
    s.push()
    s.add(Not(R))

    print("Propositional Logic Syllogism with Z3:")
    print("--------------------------------------")
    if s.check() == unsat:
        print("The conclusion (R) follows from the premises.")
    else:
        print(
            "The conclusion does NOT necessarily follow from the premises in propositional logic."
        )
        print("Counterexample Model:")
        print(s.model())


def predicate_logic_syllogism_z3():
    from z3 import (
        Solver,
        ForAll,
        Implies,
        Function,
        BoolSort,
        Const,
        Not,
        And,
        unsat,
        prove,
        DeclareSort,
    )

    # Declare a sort for Person
    Person = DeclareSort("Person")

    # Define variables
    x = Const("x", Person)

    # Define predicates
    H = Function("H", Person, BoolSort())  # H(x): x is human
    M = Function("M", Person, BoolSort())  # M(x): x is mortal
    G = Function("G", Person, BoolSort())  # G(x): x is Greek

    # Major premise: ∀x (H(x) ⇒ M(x))
    major_premise = ForAll([x], Implies(H(x), M(x)))

    # Minor premise: ∀x (G(x) ⇒ H(x))
    minor_premise = ForAll([x], Implies(G(x), H(x)))

    # Conclusion: ∀x (G(x) ⇒ M(x))
    conclusion = ForAll([x], Implies(G(x), M(x)))

    # Attempt to prove that the conclusion follows from the premises
    print("Predicate Logic Syllogism with Z3:")
    print("------------------------------------")
    print("Attempting to prove that the conclusion follows from the premises...")

    # Use Z3's prove function
    prove(Implies(And(major_premise, minor_premise), conclusion))


# Run the functions
propositional_logic_syllogism_z3()
print("\n")
predicate_logic_syllogism_z3()

Propositional Logic Syllogism with Z3:
--------------------------------------
The conclusion does NOT necessarily follow from the premises in propositional logic.
Counterexample Model:
[R = False, Q = True, P = True]


Predicate Logic Syllogism with Z3:
------------------------------------
Attempting to prove that the conclusion follows from the premises...
proved
